# Get cell level results
## Using manual label mask + bigfish output

In [1]:
import os
import numpy as np
import tifffile
import bigfish
import bigfish.stack as stack
import bigfish.segmentation as segmentation
import bigfish.plot as plot
import bigfish.detection as detection
import pandas as pd
import pathlib

/opt/miniconda3/envs/bigfish_env/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/opt/miniconda3/envs/bigfish_env/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.f

## Define spots and mask file directories

In [36]:
spots_dir = "./smFISH_quantification_output/rep1/"
mask_dir = "/Volumes/Jeff-exFAT/Kate_smFISH/rep1/"
npz_output_dir = "/Volumes/Jeff-exFAT/Kate_smFISH/rep1/cell-level-results/"
csv_output = "./smFISH_quantification_output/final_outpus/"

In [42]:
def get_final_csv(spots_dir, mask_dir, npz_output_dir, csv_output):
    ## Get spots array
    infiles = [x for x in os.listdir(spots_dir) if x.endswith('.npy')]

    for infile in infiles:
        spots_post_clustering = np.load(pathlib.Path(spots_dir).joinpath(infile))

        ## Get the mask (label image) file
        image_name = pathlib.Path(infile).stem
        mask_path = pathlib.Path(mask_dir).joinpath(f"{image_name[:-8]}_MA_mask.tiff")
        if os.path.exists(mask_path):
            masks = tifffile.imread(mask_path)

            ## Flatten mask channels
            masks = np.sum(masks, 0)

            ## Distribute spots
            fov_results = stack.extract_cell(
                cell_label=masks.astype(np.int64),
                ndim=3,
                rna_coord=spots_post_clustering
            )

            ## Save individual cell-level results
            for i, cell_results in enumerate(fov_results):
                output_path = pathlib.Path(npz_output_dir).joinpath(
                    f"{image_name}_results_cell_{i}.npz"
                )
                stack.save_cell_extracted(cell_results, str(output_path))
        else:
            print (f'The file {mask_path} does not exist')


    ## Create final csv output file
    in_npz = [x for x in os.listdir(npz_output_dir) if x.endswith('.npz')]

    file_name = []
    total_RNAs = []
        
    for file in in_npz:
        data = np.load(os.path.join(npz_output_dir, file))
        file_name.append(file)
        total_RNAs.append(data['rna_coord'].shape[0])
            
    # Inspect dataframe
    data_tall = pd.DataFrame(list(zip(file_name, total_RNAs)),
                columns =['file_name', 'total_RNAs'])

    data_tall.to_csv(csv_output)
    print('done')


## 72 hpi 3 reps

In [47]:
## Rep 1
get_final_csv(
    spots_dir ="./smFISH_quantification_output/72hpi/rep1/",
    mask_dir ="/Volumes/Jeff-exFAT/Kate_smFISH/rep1/",
    npz_output_dir ="/Volumes/Jeff-exFAT/Kate_smFISH/rep1/",
    csv_output ="./smFISH_quantification_output/final_outpus/72hpi_rep1.csv"
)
## Rep 2
get_final_csv(
    spots_dir ="./smFISH_quantification_output/72hpi/rep2/",
    mask_dir ="/Volumes/Jeff-exFAT/Kate_smFISH/rep2/",
    npz_output_dir ="/Volumes/Jeff-exFAT/Kate_smFISH/rep2/",
    csv_output ="./smFISH_quantification_output/final_outpus/72hpi_rep2.csv"
)
## Rep 3
get_final_csv(
    spots_dir ="./smFISH_quantification_output/72hpi/rep3/",
    mask_dir ="/Volumes/Jeff-exFAT/Kate_smFISH/rep3/",
    npz_output_dir ="/Volumes/Jeff-exFAT/Kate_smFISH/rep3/",
    csv_output ="./smFISH_quantification_output/final_outpus/72hpi_rep3.csv"
)

The file /Volumes/Jeff-exFAT/Kate_smFISH/rep1/20221123_221123_smFISH_72hpi_WT_no_virus_007_MA_mask.tiff does not exist
done
done
done
